In [20]:
from transformers import AutoModelForMaskedLM, AutoTokenizer, AutoConfig, AdapterConfig

In [139]:
model = AutoModelForMaskedLM.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [141]:
adapter_config = AdapterConfig.load("houlsby", non_linearity="gelu", reduction_factor=12)
model.add_adapter("test_sondre", config=adapter_config)

In [142]:
model.config.adapters.adapters

{'test_sondre': '5792d51e3161642d'}

In [103]:
model.train_adapter(["test_sondre"])

In [147]:
model.set_active_adapters(["test_sondre"])

In [148]:
model.freeze_model(False)

In [149]:
for param in model.base_model.encoder.layer[2].output.adapters.parameters():
    i = 0
    while i < 10:
        i += 1
        print(param.requires_grad)
    

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
